In [1]:
import sys
import os
project_dir = os.path.dirname(os.getcwd())
sys.path.append(project_dir) #notebooks can't look into source unless you append the parent project directory to the path
print(project_dir)

/home/ubuntu/courtney/usal_echo


In [2]:
sys.path.append('../src')
from d00_utils.db_utils import *
from d02_intermediate.create_seg_view import *
from d04_segmentation.create_probabilities_textfile_testing import *

/home/ubuntu/anaconda3/envs/usal_echo/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [3]:
#create_probabilities_textfile_testing()

In [4]:
io_clean = dbReadWriteClean()
io_views = dbReadWriteViews()

# 1. Import a_modvolume, a_measgraphref, a_measgraphic,
#        measurement_abstract_rpt
a_modvolume_df = io_clean.get_table("a_modvolume")
a_measgraphref_df = io_clean.get_table("a_measgraphref")
a_measgraphic_df = io_clean.get_table("a_measgraphic")
measurement_abstract_rpt_df = io_clean.get_table("measurement_abstract_rpt")
    

/home/ubuntu/anaconda3/envs/usal_echo/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning: Columns (8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [5]:
# 2. merge a_measgraphref_df, measurement_abstract_rpt_df
df = pd.merge(
    a_measgraphref_df,
    measurement_abstract_rpt_df,
    how="left",
    on=["studyidk", "measabstractnumber"],
)
del a_measgraphref_df
del measurement_abstract_rpt_df


In [6]:
df_2 = pd.merge(df, a_measgraphic_df, how="left", on=["instanceidk", "indexinmglist"])
df_2.count()

studyidk               858255
measabstractnumber     858255
meassequence           858255
instanceidk            858255
indexinmglist          858255
value_x                858255
howentered             858255
srinstanceidk          858255
imagesopinstanceuid    858255
measurementuid         858255
name                   858255
value_y                858255
unitname               858255
frame                  823833
graphictoolidk         823833
longaxisindex          823833
measidk                823833
loopidk                823833
instancerecordtype     823833
dtype: int64

In [7]:

#3. #create a dictionary to link measurements to views

measurement_2_view_dict = {'AVItd ap4' :'A4C', 'DVItd ap4' : 'A4C', 'VTD(el-ps4)': 'A4C', 'VTD(MDD-ps4)': 'A4C', 
                           'AVIts ap4': 'A4C', 'DVIts ap4':'A4C', 'VTS(el-ps4)':'A4C', 'VTS(MDD-ps4)':'A4C', 
                           'AVItd ap2':'A2C', 'DVItd ap2':'A2C', 'VTD(el-ps2)':'A2C', 'VTD(MDD-ps2)':'A2C', 
                           'AVIts ap2':'A2C', 'DVIts ap2':'A2C', 'VTS(el-ps2)': 'A2C', 'VTS(MDD-ps2)':'A2C', 
                           'Vol. AI (MOD-sp4)':'A4C', 'Vol. AI (MOD-sp2)':'A2C'}

measurement_2_chamber_dict = {'AVItd ap4' :'lv', 'DVItd ap4' : 'lv', 'VTD(el-ps4)': 'lv', 'VTD(MDD-ps4)': 'lv', 
                           'AVIts ap4': 'lv', 'DVIts ap4':'lv', 'VTS(el-ps4)':'lv', 'VTS(MDD-ps4)':'lv', 
                           'AVItd ap2':'lv', 'DVItd ap2':'lv', 'VTD(el-ps2)':'lv', 'VTD(MDD-ps2)':'lv', 
                           'AVIts ap2':'lv', 'DVIts ap2':'lv', 'VTS(el-ps2)': 'lv', 'VTS(MDD-ps2)':'lv', 
                           'Vol. AI (MOD-sp4)':'la', 'Vol. AI (MOD-sp2)':'la'}

measurement_2_cardio_moment_dict = {'AVItd ap4' :'ED', 'DVItd ap4' : 'ED', 'VTD(el-ps4)': 'ED', 'VTD(MDD-ps4)': 'ED', 
                           'AVIts ap4': 'ES', 'DVIts ap4':'ES', 'VTS(el-ps4)':'ES', 'VTS(MDD-ps4)':'ES', 
                           'AVItd ap2':'ED', 'DVItd ap2':'ED', 'VTD(el-ps2)':'ED', 'VTD(MDD-ps2)':'ED', 
                           'AVIts ap2':'ES', 'DVIts ap2':'ES', 'VTS(el-ps2)': 'ES', 'VTS(MDD-ps2)':'ES', 
                           'Vol. AI (MOD-sp4)':'ES', 'Vol. AI (MOD-sp2)':'ES'}



In [8]:
#4. # Add these dictionaries to the df_2
df_2['view'] = df_2['name'].map(measurement_2_view_dict)
df_2['chamber'] = df_2['name'].map(measurement_2_chamber_dict)
df_2['cardio_moment'] = df_2['name'].map(measurement_2_cardio_moment_dict)

del df



In [9]:
#5# Cut out the rows without view, chamber and cadio_moment
df_3 = df_2[pd.notna(df_2['view']) == True]
del df_2

#6.  dropping -1 instances
df_4 = df_3[df_3['instanceidk'] != -1]



In [10]:
#7. Only colums we need &drop duplicates
df_5 = df_4[['studyidk','indexinmglist', 'instanceidk', 'frame','view','chamber', 'cardio_moment']].copy()
df_6 = df_5.drop_duplicates()

del df_4
del df_5

# 8. Drop instance id 1789286 and 3121715 due to problems linking and
#        conflicts
#   Drop unnessecary row_id from orginal tables (causes problems in dbWrite)
df_7 = df_6.drop(
    df_6[(df_6.instanceidk == 1789286) | (df_6.instanceidk == 3121715)].index
)
del df_6


In [11]:
instances_unique_master_list = io_views.get_table('instances_unique_master_list')
instances_unique_master_list.head()

,sopinstanceuid,instancefilename,instanceidk,studyidk
0,1.2.840.113543.6.6.4.6.62554526370968849932129...,6RYXZNZW,57157,2882
1,1.2.840.113543.6.6.4.6.64175920513291079284612...,6RYXZO03,57158,2882
2,1.2.840.113543.6.6.4.6.63515355813678132433121...,6RYXZO0D,57159,2882
3,1.2.840.113543.6.6.4.6.63385835434171778347132...,6RYXZO0K,57160,2882
4,1.2.840.113543.6.6.4.6.62250778798156715321013...,6RYXZO0Y,57161,2882


In [12]:
# Adding file name

df_8 = pd.merge(df_7, instances_unique_master_list, how="left", 
                on=["instanceidk", "studyidk"])    
#del instances_unique_master_list

# below cleans the filename field
df_8["instancefilename"] = df_8["instancefilename"].apply(lambda x: str(x).strip())

# 9. Saving to frames_by_volume_mask
df_8.columns = map(str.lower, df_8.columns)
#io_views.save_to_db(df_8, "frames_by_volume_mask")

# 9. merge with a_modvolume
#df_9 = pd.merge(
#    a_modvolume_df, df_8, how="left", on=["instanceidk", "indexinmglist"]
#)
#del df_8
#del a_modvolume_df

#io_views.save_to_db(df_9, "chords_by_volume_mask")
    

In [13]:
df_8.head()

,studyidk,indexinmglist,instanceidk,frame,view,chamber,cardio_moment,sopinstanceuid,instancefilename
0,2882,0,57164,8.0,A4C,lv,ES,1.2.840.113543.6.6.4.6.61287632529377935401711...,6RYXZO20
1,2882,1,57164,15.0,A4C,lv,ED,1.2.840.113543.6.6.4.6.61287632529377935401711...,6RYXZO20
2,2966,0,58207,1.0,A4C,lv,ES,1.2.840.113543.6.6.4.6.62432989118278372356313...,7QVURND6
3,2966,1,58207,8.0,A4C,lv,ED,1.2.840.113543.6.6.4.6.62432989118278372356313...,7QVURND6
4,5898,0,102950,17.0,A4C,lv,ED,1.2.840.113543.6.6.4.6.61767879315804788584108...,FN8YYXBS


In [15]:
df_8.shape

(72485, 9)